In [1]:
# Import Library
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from kafka import KafkaConsumer, TopicPartition

In [2]:
# Load .env file
dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
# Load .env variables
kafka_host = os.getenv('KAFKA_HOST')
kafka_topic = os.getenv('KAFKA_TOPIC_ASSIGNMENT')

In [4]:
def safe_deserializer(m):
    if isinstance(m, bytes):
        m = m.decode('utf-8')
    return json.loads(m)

consumer_1 = KafkaConsumer(
    kafka_topic,
    group_id='assignment-group',
    bootstrap_servers=[f'{kafka_host}:9092'],
    value_deserializer=safe_deserializer,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    consumer_timeout_ms=45000
)

In [5]:
# consumer_1.seek_to_beginning()

In [6]:
print("⏳ Listening for login events...")

login_stats = {}
total_messages = 0
first = True
topic = None
partition = None

for i, message in enumerate(consumer_1):
    
    if first:
        print(f"Consuming data from topic '{message.topic}' partition {message.partition}")
        topic = message.topic
        partition = message.partition
        first = False
        
    event = message.value
    role = event["role"]
    success = event["login_success"]

    if role not in login_stats:
        login_stats[role] = {"success": 0, "failed": 0}

    if success:
        login_stats[role]["success"] += 1
    else:
        login_stats[role]["failed"] += 1

    print(f"[{role}] Success: {login_stats[role]['success']} | Failed: {login_stats[role]['failed']}")
    total_messages += 1
    
print(f"\nFinished consuming {total_messages} messages from topic '{topic}' partition {partition}'")

⏳ Listening for login events...
Consuming data from topic 'assignment_topic' partition 0
[Guest] Success: 1 | Failed: 0
[User] Success: 1 | Failed: 0
[Admin] Success: 1 | Failed: 0
[Guest] Success: 2 | Failed: 0
[Guest] Success: 3 | Failed: 0
[User] Success: 2 | Failed: 0
[Guest] Success: 3 | Failed: 1
[Admin] Success: 1 | Failed: 1
[Guest] Success: 3 | Failed: 2
[Admin] Success: 2 | Failed: 1
[Guest] Success: 3 | Failed: 3
[User] Success: 2 | Failed: 1

Finished consuming 12 messages from topic 'assignment_topic' partition 0'
